In [1]:
import math
import time
import pandas as pd
import base64 as b64
import os
import numpy as np
import re
import requests
from bs4 import BeautifulSoup

# Uncomment when ruunning on colab
# from google.colab import drive
# drive.mount('/content/gdrive')
# os.chdir('/content/gdrive/My Drive/733')

Coordinates list

In [2]:
url = "https://api2.realtor.ca/Listing.svc/PropertySearch_Post"
raw_csv_name = 'realtor_properties_raw.csv'
details_csv_name = 'realtor_properties_details.csv'
details_web_csv_name = 'realtor_properties_details_web.csv'
final_csv_name = 'realtor_properties.csv'

# Co-ordinates data picked from realtor
coordinates_dict = [
    {"LatitudeMax": 54.8049024, "LongitudeMax": -117.9861018, "LatitudeMin": 49.4157337, "LongitudeMin": -144.3576854},
    {"LatitudeMax": 49.4001858, "LongitudeMax": -122.8600240, "LatitudeMin": 48.6800156, "LongitudeMin": -126.1564720},
    {"LatitudeMax": 49.2209661, "LongitudeMax": -121.6302292, "LatitudeMin": 48.8608805, "LongitudeMin": -123.2784532},
    {"LatitudeMax": 49.3318846, "LongitudeMax": -122.4158891, "LatitudeMin": 49.1525708, "LongitudeMin": -123.2400011},
    {"LatitudeMax": 49.2434697, "LongitudeMax": -122.3857596, "LatitudeMin": 49.0638345, "LongitudeMin": -123.2098716},
    {"LatitudeMax": 49.5250750, "LongitudeMax": -121.9393713, "LatitudeMin": 49.1672043, "LongitudeMin": -123.5875953},
    {"LatitudeMax": 49.5859841, "LongitudeMax": -123.0319840, "LatitudeMin": 48.1406349, "LongitudeMin": -129.6248799}
]

Method to generate property listing of different coordinates

In [3]:
def get_properties_listing(property_type_arg, lat_lon_dict):
    property_type_identifier = 'residential' if property_type_arg['PropertyTypeGroupID'] == "1" else 'Commercial'
    max_results = 200
    current_page = 1
    coordinate_dataset = pd.DataFrame()
    while current_page > 0:
        print('Current Page {}$'.format(current_page))
        payload = {
            "CultureId": "1",
            "ApplicationId": "1",
            "RecordsPerPage": max_results,
            "PropertyTypeGroupID": property_type_arg['PropertyTypeGroupID'],
            "PropertySearchTypeId": property_type_arg['PropertySearchTypeId'],
            "PriceMin": 0,
            "PriceMax": 0,
            "LandSizeRange": "0-0",
            "TransactionTypeId": "1",
            "StoreyRange": "0-0",
            "BedRange": "0-0",
            "BathRange": "0-0",
            "LongitudeMin": lat_lon_dict['LongitudeMin'],
            "LongitudeMax": lat_lon_dict['LongitudeMax'],
            "LatitudeMin": lat_lon_dict['LatitudeMin'],
            "LatitudeMax": lat_lon_dict['LatitudeMax'],
            "SortOrder": "A",
            "SortBy": "1",
            "ZoomLevel": "11",
            "CurrentPage": current_page,
            "Version": "7.0"
        }
        headers = {'content-type': 'application/x-www-form-urlencoded'}
        while True:
            try:
                r = requests.post(url, data=payload, headers=headers)
                print(r)
                break
            except ConnectionError:
                time.sleep(1)
                print('Connection Fail...')
                pass
        if r.ok:
            results = r.json()['Results']
            print('Found {} results!'.format(len(results)))
            if len(results) == max_results:
                current_page = current_page + 1
                # current_page = -1
            else:
                current_page = -1
            try:
                for result in results:
                    price = result['Property'].get('Price', '')
                    coordinate_dataset = coordinate_dataset.append({
                        'lng': result['Property']['Address']['Longitude'],
                        'lat': result['Property']['Address']['Latitude'],
                        'address': result['Property']['Address']['AddressText'],
                        'postal_code': result['PostalCode'],
                        'property_type': result['Property']['Type'],
                        'price': price if price != '' else result['Property'].get('LeaseRent', ''),
                        'ownership_type': result['Property'].get('OwnershipType', ''),
                        'mls': result['MlsNumber'],
                        'bathrooms': result['Building'].get('BathroomTotal', 0),
                        'bedrooms': result['Building'].get('Bedrooms', 0),
                        'type': result['Building'].get('Type', ''),
                        'ammenities': result['Building'].get('Ammenities', ''),
                        'land_size': result['Land'].get('SizeTotal', 0),
                        'land_size_interior': result['Building'].get('SizeInterior', 0),
                        'absoluteUrl': 'https://www.realtor.ca' + result['RelativeDetailsURL'],
                        'property_type_identifier': property_type_identifier,
                        'type_id': result['Property']['TypeId']
                    }, ignore_index=True)
            except KeyError:
                return print(KeyError)
    return coordinate_dataset

In [ ]:
def get_properties_details(realtor_properties_df_arg):
    realtor_properties_details_df = pd.DataFrame()
    total_properties = len(realtor_properties_df_arg)
    empty_value_dict = {'value': 0}

    # creating temp csv files and storing the intermediate results to them
    num_records_per_csv = math.ceil(total_properties/10)
    current_file = 0
    part_csv_files_count = math.ceil(total_properties / num_records_per_csv)

    index_to_start_from = 0
    for index in range(0, part_csv_files_count):
        tmp_csv_file_name = '{}/rpd_tmp_{}.csv'.format(os.getcwd(), index)
        if os.path.exists(tmp_csv_file_name):
            index_to_start_from += num_records_per_csv
            current_file = index + 1
        else:
            index_to_start_from = index * num_records_per_csv
            break

    # Filter the main DF and start the process of getting the details again for the remaining number of results.
    realtor_properties_df_arg = realtor_properties_df_arg[index_to_start_from:]
    total_properties_to_process = len(realtor_properties_df_arg)
    actual_index = 0

    if index_to_start_from < total_properties:
        for index, realtor_property in realtor_properties_df_arg.iterrows():
            mls_number = realtor_property['mls']
            print('Getting details for {} record {} out of {}'.format(mls_number, (actual_index + 1), total_properties_to_process))
            url_params = '{"token":"b7505b4db3e235cb1a0430fb0038e8da97b3257a6376a5085a2dfed7911ffdfd38164eab0b97340d",' \
                         '"lat":"' + str(realtor_property['lat']) + '","lng":"' + str(realtor_property['lng']) + '"}'
            encoded_url_param = b64.b64encode(url_params.encode())
            details_url = "https://api.locallogic.co/v1/scores/bG9jYWxsb2dpYw{}".format(encoded_url_param.decode())

            high_schools = ''
            primary_schools = ''
            transit_friendly = ''
            groceries = ''
            restaurants = ''
            pedestrian_friendly = ''
            cycling_friendly = ''
            car_friendly = ''
            vibrant = ''
            shopping = ''
            daycares = ''
            nightlife = ''
            cafes = ''
            quiet = ''
            parks = ''

            while True:
                try:
                    details_request = requests.get(details_url)
                    break
                except ConnectionError:
                    time.sleep(1)
                    print('Connection Fail...')
                    pass
            if details_request.ok:
                results = details_request.json()['data']['attributes']

                high_schools = results.get('high_schools', empty_value_dict).get('value')
                primary_schools = results.get('primary_schools', empty_value_dict).get('value')
                transit_friendly = results.get('transit_friendly', empty_value_dict).get('value')
                groceries = results.get('groceries', empty_value_dict).get('value')
                restaurants = results.get('restaurants', empty_value_dict).get('value')
                pedestrian_friendly = results.get('pedestrian_friendly', empty_value_dict).get('value')
                cycling_friendly = results.get('cycling_friendly', empty_value_dict).get('value')
                car_friendly = results.get('car_friendly', empty_value_dict).get('value')
                vibrant = results.get('vibrant', empty_value_dict).get('value')
                shopping = results.get('shopping', empty_value_dict).get('value')
                daycares = results.get('daycares', empty_value_dict).get('value')
                nightlife = results.get('nightlife', empty_value_dict).get('value')
                cafes = results.get('cafes', empty_value_dict).get('value')
                quiet = results.get('quiet', empty_value_dict).get('value')
                parks = results.get('parks', empty_value_dict).get('value')

            realtor_properties_details_df = realtor_properties_details_df \
                .append({'lng': realtor_property['lng'],
                         'lat': realtor_property['lat'],
                         'address': realtor_property['address'],
                         'postal_code': realtor_property['postal_code'],
                         'property_type': realtor_property['property_type'],
                         'price': realtor_property['price'],
                         'ownership_type': realtor_property['ownership_type'],
                         'mls': mls_number,
                         'bathrooms': realtor_property['bathrooms'],
                         'bedrooms': realtor_property['bedrooms'],
                         'type': realtor_property['type'],
                         'ammenities': realtor_property['ammenities'],
                         'land_size': realtor_property['land_size'],
                         'land_size_interior': realtor_property['land_size_interior'],
                         'absoluteUrl': realtor_property['absoluteUrl'],
                         'property_type_identifier': realtor_property['property_type_identifier'],
                         'type_id': realtor_property['type_id'],
                         'high_schools': high_schools,
                         'primary_schools': primary_schools,
                         'transit_friendly': transit_friendly,
                         'groceries': groceries,
                         'restaurants': restaurants,
                         'pedestrian_friendly': pedestrian_friendly,
                         'cycling_friendly': cycling_friendly,
                         'car_friendly': car_friendly,
                         'vibrant': vibrant,
                         'shopping': shopping,
                         'daycares': daycares,
                         'nightlife': nightlife,
                         'cafes': cafes,
                         'quiet': quiet,
                         'parks': parks,
                         }, ignore_index=True)

            # try to save part results into the temp csv
            if ((actual_index + 1) % num_records_per_csv) == 0:
                tmp_file_name = 'rpd_tmp_{}.csv'.format(current_file)
                realtor_properties_details_df.to_csv(tmp_file_name)
                current_file += 1
                realtor_properties_details_df = pd.DataFrame()
                print('File with {} records {} out of {}'.format(tmp_file_name, (actual_index + 1), total_properties_to_process))
            actual_index += 1

        # Write the remaining DF to storage
        if len(realtor_properties_details_df) > 0:
            tmp_file_name = 'rpd_tmp_{}.csv'.format(current_file)
            realtor_properties_details_df.to_csv(tmp_file_name)

    realtor_bc_df = pd.DataFrame()
    for index in range(0, part_csv_files_count):
        realtor_bc_df = pd.concat(
            [realtor_bc_df, pd.read_csv('rpd_tmp_{}.csv'.format(index))], ignore_index=False
        )

    realtor_bc_df.to_csv(details_csv_name)

In [ ]:
def get_div_value(div):
    value_div = div.findAll("div", class_="propertyDetailsSectionContentValue")
    value_text = ''
    if len(value_div) > 0:
        value_text = value_div[0].text
    return value_text


def get_properties_details_web(realtor_properties_df_arg):
    realtor_properties_details_df = pd.DataFrame()
    headers = {'Cookie': 'TermsOfUseAgreement=2018-06-07;'}

    total_properties = len(realtor_properties_df_arg)
    num_records_per_csv = math.ceil(total_properties / 10)
    current_file = 0
    part_csv_files_count = math.ceil(total_properties / num_records_per_csv)

    index_to_start_from = 0
    for index in range(0, part_csv_files_count):
        tmp_csv_file_name = '{}/rpdw_tmp_{}.csv'.format(os.getcwd(), index)
        if os.path.exists(tmp_csv_file_name):
            index_to_start_from += num_records_per_csv
            current_file = index + 1
        else:
            index_to_start_from = index * num_records_per_csv
            break

    if index_to_start_from < total_properties:
        realtor_properties_df_arg = realtor_properties_df_arg[index_to_start_from:]
        total_properties_to_process = len(realtor_properties_df_arg)
        actual_index = 0

        for index, realtor_property in realtor_properties_df_arg.iterrows():
            print('Details {} out of {} for {}'.format(actual_index, total_properties_to_process, realtor_property['absoluteUrl']))
            request = requests.get(realtor_property['absoluteUrl'], headers=headers)
            soup = BeautifulSoup(request.text, "html.parser")

            try:
                div_building_type = soup.findAll("div", id="propertyDetailsSectionContentSubCon_BuildingType")
                div_num_stories = soup.findAll("div", id="propertyDetailsSectionContentSubCon_Stories")
                div_title = soup.findAll("div", id="propertyDetailsSectionContentSubCon_Title")
                div_land_size = soup.findAll("div", id="propertyDetailsSectionContentSubCon_LandSize")
                div_built_in = soup.findAll("div", id="propertyDetailsSectionContentSubCon_BuiltIn")
                div_property_tax = soup.findAll("div", id="propertyDetailsSectionContentSubCon_AnnualPropertyTaxes")
                div_parking_space = soup.findAll("div", id="propertyDetailsSectionContentSubCon_TotalParkingSpaces")

                realtor_properties_details_df = realtor_properties_details_df.append(realtor_property)

                realtor_properties_details_df.at[index, 'building_type'] = get_div_value(div_building_type[0]) \
                    if (len(div_building_type) > 0) else ''
                realtor_properties_details_df.at[index, 'num_stories'] = get_div_value(div_num_stories[0]) \
                    if (len(div_num_stories) > 0) else ''
                realtor_properties_details_df.at[index, 'title'] = get_div_value(div_title[0]) \
                    if (len(div_title) > 0) else ''
                realtor_properties_details_df.at[index, 'land_size'] = get_div_value(div_land_size[0]) \
                    if (len(div_land_size) > 0) else ''
                realtor_properties_details_df.at[index, 'built_in'] = get_div_value(div_built_in[0]) \
                    if (len(div_built_in) > 0) else ''
                realtor_properties_details_df.at[index, 'property_tax'] = get_div_value(div_property_tax[0]) \
                    if (len(div_property_tax) > 0) else ''
                realtor_properties_details_df.at[index, 'parking_space'] = get_div_value(div_parking_space[0]) \
                    if (len(div_parking_space) > 0) else ''
            except Exception as e:
                print(e)
                continue

                # try to save part results into the temp csv
            if ((actual_index + 1) % num_records_per_csv) == 0:
                tmp_file_name = 'rpdw_tmp_{}.csv'.format(current_file)
                realtor_properties_details_df.to_csv(tmp_file_name)
                current_file += 1
                realtor_properties_details_df = pd.DataFrame()
                print('File with {} records {} out of {}'.format(tmp_file_name, (actual_index + 1),
                                                                 total_properties_to_process))
            actual_index += 1

        # Write the remaining DF to storage
        if len(realtor_properties_details_df) > 0:
            tmp_file_name = 'rpdw_tmp_{}.csv'.format(current_file)
            realtor_properties_details_df.to_csv(tmp_file_name)

    realtor_bc_df = pd.DataFrame()
    for index in range(0, part_csv_files_count):
        realtor_bc_df = pd.concat(
            [realtor_bc_df, pd.read_csv('rpdw_tmp_{}.csv'.format(index))], ignore_index=False
        )
    realtor_bc_df.to_csv(details_web_csv_name)

Method to get properties details

# Sanitize Realtor Properties Data

In [ ]:
def sanitize_realtor_properties_csv(realtor_properties_df_arg):
    # drop all the records where the property price is monthly
    clean_df = realtor_properties_df_arg[~realtor_properties_df_arg['price'].str.contains('Monthly')]
    clean_df = clean_df.reset_index(drop=True)
    clean_df[['size', 'unit']] = clean_df['land_size_interior'].str.split(" ", expand=True)
    clean_df['postal_code'] = clean_df['postal_code'].str.upper()
    clean_df['postal_code'] = clean_df.postal_code.str.slice(stop=3)
    clean_df['price'] = clean_df['price'].replace('$', '')
    # Adding dummy column to the DF so we can populate it afterwards
    clean_df['total_price'] = np.nan
    clean_df.assign(total_price=np.nan)

    for i in range(len(clean_df['price'])):
        # check the properties where the price is per square feet get total price by multiplying price and size
        if 'feet' in clean_df.iloc[i]['price'].split():
            clean_df.iloc[i, clean_df.columns.get_loc('price')] = re.sub('\\D', '', clean_df.iloc[i]['price'])

            price = float(clean_df.iloc[i]['price'])
            size = float(clean_df.iloc[i]['size'])
            clean_df.iloc[i, clean_df.columns.get_loc('total_price')] = price * size
        else:
            # the price in square feet so we can use the price as it is in total_price column
            clean_df.iloc[i, clean_df.columns.get_loc('price')] = re.sub('\\D', '', clean_df.iloc[i]['price'])
            clean_df.iloc[i, clean_df.columns.get_loc('total_price')] = int(clean_df.iloc[i]['price'])
    return clean_df

# Calling the Functions for Property list (Coordinate CSV Creation)

In [ ]:
realtor_properties_df = pd.DataFrame()

    if not os.path.isfile(raw_csv_name):
        for i in range(len(coordinates_dict)):
            print('Getting results for {}'.format(coordinates_dict[i]))

            # First one is Residential, Second one is Commercial
            property_types = [{"PropertyTypeGroupID": "1", "PropertySearchTypeId": "1"},
                              {"PropertyTypeGroupID": "2", "PropertySearchTypeId": "0"}]

            for property_type in property_types:
                df = get_properties_listing(property_type, coordinates_dict[i])
                realtor_properties_df = pd.concat([realtor_properties_df, df], ignore_index=True)

        print('Total results fetched {}'.format(len(realtor_properties_df)))
        realtor_properties_df = realtor_properties_df.drop_duplicates('mls')
        print('Total results after dropping duplicates {}'.format(len(realtor_properties_df)))

        realtor_properties_df.to_csv(raw_csv_name)
    else:
        realtor_properties_df = pd.read_csv(raw_csv_name)

    # Dropping the results which are not in the Metro Vancouver
    print('Total results before filtering {}'.format(len(realtor_properties_df)))
    accepted_zip_codes = ['V3', 'V4', 'V5', 'V6', 'V7']
    realtor_properties_df = realtor_properties_df[realtor_properties_df['postal_code'].str[:2].isin(accepted_zip_codes)]
    print('Total results after filtering {}'.format(len(realtor_properties_df)))

    if not os.path.isfile(final_csv_name):
        realtor_properties_df = sanitize_realtor_properties_csv(realtor_properties_df)
        realtor_properties_df.to_csv(final_csv_name)
    else:
        realtor_properties_df = pd.read_csv(final_csv_name)

    if not os.path.isfile(details_csv_name):
        get_properties_details(realtor_properties_df)

    if not os.path.isfile(details_web_csv_name):
        get_properties_details_web(realtor_properties_df)
    else:
        print('All tasks done successfully')

Getting results for {'LatitudeMax': 54.8049024, 'LongitudeMax': -117.9861018, 'LatitudeMin': 49.4157337, 'LongitudeMin': -144.3576854}
Current Page 1$
<Response [200]>
Found 200 results!
Current Page 2$
<Response [200]>
Found 200 results!
Current Page 3$
<Response [200]>
Found 200 results!
Current Page 4$
<Response [200]>
Found 200 results!
Current Page 5$
<Response [200]>
Found 200 results!
Current Page 6$
<Response [200]>
Found 200 results!
Current Page 7$
<Response [200]>
Found 200 results!
Current Page 8$
<Response [200]>
Found 200 results!
Current Page 9$
<Response [200]>
Found 200 results!
Current Page 10$
<Response [200]>
Found 200 results!
Current Page 11$
<Response [200]>
Found 200 results!
Current Page 12$
<Response [200]>
Found 200 results!
Current Page 13$
<Response [200]>
Found 200 results!
Current Page 14$
<Response [200]>
Found 200 results!
Current Page 15$
<Response [200]>
Found 200 results!
Current Page 16$
<Response [200]>
Found 200 results!
Current Page 17$
<Response

Current Page 28$
<Response [200]>
Found 200 results!
Current Page 29$
<Response [200]>
Found 200 results!
Current Page 30$
<Response [200]>
Found 200 results!
Current Page 31$
<Response [200]>
Found 200 results!
Current Page 32$
<Response [200]>
Found 200 results!
Current Page 33$
<Response [200]>
Found 200 results!
Current Page 34$
<Response [200]>
Found 200 results!
Current Page 35$
<Response [200]>
Found 200 results!
Current Page 36$
<Response [200]>
Found 200 results!
Current Page 37$
<Response [200]>
Found 200 results!
Current Page 38$
<Response [200]>
Found 200 results!
Current Page 39$
<Response [200]>
Found 200 results!
Current Page 40$
<Response [200]>
Found 200 results!
Current Page 41$
<Response [200]>
Found 200 results!
Current Page 42$
<Response [200]>
Found 200 results!
Current Page 43$
<Response [200]>
Found 200 results!
Current Page 44$
<Response [200]>
Found 200 results!
Current Page 45$
<Response [200]>
Found 200 results!
Current Page 46$
<Response [200]>
Found 200 re

Current Page 5$
<Response [200]>
Found 200 results!
Current Page 6$
<Response [200]>
Found 200 results!
Current Page 7$
<Response [200]>
Found 200 results!
Current Page 8$
<Response [200]>
Found 200 results!
Current Page 9$
<Response [200]>
Found 200 results!
Current Page 10$
<Response [200]>
Found 200 results!
Current Page 11$
<Response [200]>
Found 200 results!
Current Page 12$
<Response [200]>
Found 200 results!
Current Page 13$
<Response [200]>
Found 200 results!
Current Page 14$
<Response [200]>
Found 200 results!
Current Page 15$
<Response [200]>
Found 200 results!
Current Page 16$
<Response [200]>
Found 200 results!
Current Page 17$
<Response [200]>
Found 200 results!
Current Page 18$
<Response [200]>
Found 200 results!
Current Page 19$
<Response [200]>
Found 200 results!
Current Page 20$
<Response [200]>
Found 200 results!
Current Page 21$
<Response [200]>
Found 200 results!
Current Page 22$
<Response [200]>
Found 200 results!
Current Page 23$
<Response [200]>
Found 200 results

Got details for 449012
Got details for 149916
Got details for 148004
Got details for R2332734
Got details for 450171
Got details for R2343464
Got details for R2295983
Got details for 148245
Got details for 10175674
Got details for R2334037
Got details for 447590
Got details for 148106
Got details for R2312317
Got details for 149432
Got details for 146646
Got details for R2327964
Got details for 176695
Got details for 14175
Got details for 149777
Got details for 148354
Got details for 149928
Got details for R2337318
Got details for 10175578
Got details for 10176006
Got details for 449692
Got details for 10164259
Got details for 10172680
Got details for 10150709
Got details for 10169849
Got details for R2318192
Got details for R2260064
Got details for R2272101
Got details for 148473
Got details for 428197
Got details for 148876
Got details for 48124
Got details for 10175818
Got details for R2341382
Got details for R2310652
Got details for R2337279
Got details for R2342055
Got details for

Got details for 149742
Got details for 148631
Got details for 147637
Got details for 148744
Got details for R2310841
Got details for R2249431
Got details for R2303909
Got details for R2315040
Got details for R2302448
Got details for R2328487
Got details for 48745
Got details for R2294901
Got details for 148400
Got details for 149554
Got details for 10173941
Got details for 147489
Got details for 47944
Got details for 48349
Got details for R2339295
Got details for R2321789
Got details for 149306
Got details for 10175415
Got details for 145850
Got details for R2138994
Got details for R2303179
Got details for 47362
Got details for R2342068
Got details for 448038
Got details for 148872
Got details for 14128
Got details for 146382
Got details for 10176082
Got details for 450809
Got details for 149118
Got details for 10168503
Got details for 10173843
Got details for 10176600
Got details for 10175668
Got details for 445688
Got details for 449860
Got details for 10173845
Got details for 449115

...

Got details for R2343459
Got details for R2320152
Got details for R2335861
Got details for R2311804
Got details for R2323926
Got details for R2331553
Got details for R2318731
Got details for R2317404
Got details for R2341376
Got details for R2324848
Got details for R2339825
Got details for R2343477
Got details for R2342860
Got details for R2336049
Got details for R2341318
Got details for R2330292
Got details for R2327283
Got details for R2327284
Got details for R2333152
Got details for R2301763
Got details for R2343761
Got details for R2327161
Got details for R2333187
Got details for R2327949
Got details for R2329057
Got details for R2297877
Got details for R2337552
Got details for R2332491
Got details for R2329787
Got details for R2327140
Got details for R2331313
Got details for R2309888
Got details for R2313132
Got details for R2287099
Got details for R2333316
Got details for R2292090
Got details for R2307314
Got details for R2334986
Got details for R2296007
Got details for R2309118


Got details for 400366
Got details for 398045
Got details for 405769
Got details for 404396
Got details for 405405
Got details for 402019
Got details for 404940
Got details for 401623
Got details for 404253
Got details for 398046
Got details for 401031
Got details for 405914
Got details for 404594
Got details for 400637
Got details for 400095
Got details for 405558
Got details for R2303635
Got details for 398069
Got details for R2319469
Got details for 449795
Got details for R2343246
Got details for 405926
Got details for 402252
Got details for 447139
Got details for 404931
Got details for 399604
Got details for R2320507
Got details for 404793
Got details for 405528
Got details for 405362
Got details for 405267
Got details for 404921
Got details for 402039
Got details for 450643
Got details for 404865
Got details for 402354
Got details for 402294
Got details for 404904
Got details for 405616
Got details for 405488
Got details for 405805
Got details for 404448
Got details for 405800
Got

Got details for 401960
Got details for 405941
Got details for 404559
Got details for 405880
Got details for 404966
Got details for 405739
Got details for 401882
Got details for 402330
Got details for 404302
Got details for 401233
Got details for 404823
Got details for 404249
Got details for 404248
Got details for 405261
Got details for 405355
Got details for 402006
Got details for 402008
Got details for 401939
Got details for 405749
Got details for 405887
Got details for R2236262
Got details for 405551
Got details for 448580
Got details for 449725
Got details for 405442
Got details for R2236224
Got details for 402267
Got details for 400063
Got details for 390855
Got details for 404799
Got details for R2328902
Got details for 405832
Got details for 404387
Got details for 405807
Got details for 405936
Got details for R2333823
Got details for 404523
Got details for 405505
Got details for 400504
Got details for 405717
Got details for 405746
Got details for 405436
Got details for 405942
Got

Got details for R2324664
Got details for 402099
Got details for 405583
Got details for 402194
Got details for 404283
Got details for 405965
Got details for 404738
Got details for 405414
Got details for 401609
Got details for 405841
Got details for 405699
Got details for 405964
Got details for 405432
Got details for 448484
Got details for 405554
Got details for 448148
Got details for 441971
Got details for 404992
Got details for 402344
Got details for 400979
Got details for 405659
Got details for 397968
Got details for X4101775
Got details for 404284
Got details for 401973
Got details for 404936
Got details for 401612
Got details for 405485
Got details for 402204
Got details for 404609
Got details for 405757
Got details for 405946
Got details for 404560
Got details for 406002
Got details for 405546
Got details for 400710
Got details for R2322096
Got details for 405409
Got details for 402312
Got details for R2333481
Got details for 401331
Got details for 405719
Got details for 402228
Got

Got details for 405601
Got details for 405991
Got details for 404943
Got details for 404854
Got details for 404569
Got details for 401680
Got details for 405243
Got details for 405340
Got details for 405669
Got details for 405167
Got details for R2340122
Got details for R2308644
Got details for 399532
Got details for 405175
Got details for 399458
Got details for 399457
Got details for R2281980
Got details for R2258880
Got details for 404431
Got details for 402057
Got details for 404656
Got details for 404635
Got details for 391421
Got details for 401745
Got details for 405288
Got details for 405289
Got details for R2315931
Got details for 405464
Got details for 405176
Got details for 405178
Got details for 405177
Got details for 400743
Got details for 395539
Got details for 406026
Got details for 405189
Got details for R2307050
Got details for R2328680
Got details for 405613
Got details for 401720
Got details for 405250
Got details for 405230
Got details for 405828
Got details for 4058